# 1. Initial setup

## Required packages and databases

Our analysis requires several software packages that can be installed via `conda` using the provided environment file (`environment.yml`). Additionally, we require [USEARCH](https://www.drive5.com/usearch/manual/install.html) and `SQLite3`. The latter can be installed using `sudo apt-get install sqlite3`

### Downloading UHGG
Our analysis also requires the UHGG and UHGP-90 databases from [Almedia et al., (2020)](https://www.nature.com/articles/s41587-020-0603-3). Specifically:

1. [UHGG catalogue](https://ftp.ebi.ac.uk/pub/databases/metagenomics/mgnify_genomes/human-gut/v1.0/uhgg_catalogue/) saved in a folder called `data/uhgg/uhgg_catalogue`;
2. [UHGP-90](https://ftp.ebi.ac.uk/pub/databases/metagenomics/mgnify_genomes/human-gut/v1.0/uhgp_catalogue/uhgp-90.tar.gz) saved in `data/uhgg/uhgp-90`;
3. [UHGG Kraken DB](https://ftp.ebi.ac.uk/pub/databases/metagenomics/mgnify_genomes/human-gut/v1.0/uhgg_kraken2-db/) saved in `data/uhgg/uhgg_kraken2-db`; and
4. [All UHGG genomes](https://ftp.ebi.ac.uk/pub/databases/metagenomics/mgnify_genomes/human-gut/v1.0/all_genomes/) saved in `data/uhgg/all_genomes`.

### AlphaFold (AFDB) mappings

To map the UHGP-90 database onto AlphaFold along with AFDB clusters (see [Barrio-Hernandez et al., (2023)](https://nature.com/articles/s41586-023-06510-w)), we need the following saved in a directory called `data/uhgg/foldseek`:
1. [AFDB catalogue](https://github.com/google-deepmind/alphafold/blob/main/afdb/README.md) saved as `afdb.faa`; and
2. [AFDB clusters](https://afdb-cluster.steineggerlab.workers.dev/5-allmembers-repId-entryId-cluFlag-taxId.tsv.gz) from [Barrio-Hernandez et al., (2023)](https://nature.com/articles/s41586-023-06510-w) saved as `afdb.cluster.tsv`.

## Preprocessing

### Merging genome annotations
UHGG provides GFF, InterProScan, and KEGG annotations for genomes. For convenience, we merge these into a single annotation file using the code below:

In [ ]:
import os, sys
sys.path.append(os.path.abspath('../../mgxevo/src/'))
import mgxevo as mg

os.makedirs('data', exist_ok=True)
os.makedirs('write', exist_ok=True)

In [ ]:
%%bash -s "$mg.tl.uhgg.script_path"

# get a list of MGYGs from UHGG 
cut -f 18 data/uhgg/genomes-nr_metadata.tsv | tail -n +2 | sort -u > data/uhgg/genomes-nr.mgyg_ids.txt

# iterate script over these MGYGs
uhgg_catalogue="./data/uhgg/uhgg_catalogue/"
script_path="$1"

annotate_gene() {
    x="$1"
    ifn="${uhgg_catalogue}$(echo ${x} | cut -c 1-13)/${x}/genome/${x}.gff"
    
    # Convert gff to gtf
    agat_convert_sp_gff2gtf.pl --gff $ifn --out ${ifn/.gff/.gtf} > /dev/null 2>&1
    
    # merge annotation
    Rscript ${script_path}/4.gene_annotations_mgyg.r --MGYG_id $x --uhgg_catalogue $uhgg_catalogue
}

export -f annotate_gene
export uhgg_catalogue
export script_path

cat data/uhgg/genomes-nr.mgyg_ids.txt | parallel --progress -j360 annotate_gene

### Mapping UHGP locus IDs onto UHGP-90

Mapping a UHGP locus ID onto its UHGP-90 protein ID is computationally expensive. One way to address this is to use `SQLite3` to build an SQL database:
```
sqlite3 uhgg.sqlite
sqlite> create table map(seed,gene);
sqlite> .mode tabs
sqlite> .import data/uhgg/uhgp-90/uhgp-90.tsv map
sqlite> .schema map
```

This allows us to efficiently map the gene loci found in UHGG's presence-absence matrices (`*.genes_presence-absence_locus.csv`) onto their respective UHGP-90 protein IDs:

In [ ]:
%%bash -s "$mg.tl.uhgg.script_path"

cat data/uhgg/genomes-nr.mgyg_ids.txt | parallel --progress -j360 Rscript $1/3.map2uhgp90.r {} ./data/uhgg/

# map UHGP90 IDs to KEGG
Rscript $1/5.uhgp90-kegg.r --uhgp90_dir "./data/uhgg/uhgp90/" --output "./data/uhgg/uhgp90/'

#### Mapping UHGP-90 onto AlphaFold (AFDB) and AFDB clusters

To map UHGP-90 IDs onto AlphaFold proteins we can use the following script:

In [ ]:
%%bash -s "$mg.tl.uhgg.script_path"

python $1/6.uhgp90-afdb.py "./data/uhgg/foldseek/afdb.faa" "./data/uhgg/uhgp90/uhgp-90_hq.faa" > ./write/uhgp2afdb.tsv

The resulting mapping of UHGP-90 onto AlphaFold (`/write/uhgp2afdb.tsv`) can then be merged with the AFDB clusters (`afdb.cluster.tsv`).

### Processing metadata

Our analysis requires Table S1 (saved as `dx.csv` when importing `mgxevo`) from the manuscript along with an extra column called `file` for where the corresponding FASTA is saved. **Note:** each sample should only have _one_ entry in the `file` column. Paired reads can be denoted using a wildcard (_e.g._, `example_R*.fastq.gz` for `example_R1.fastq.gz` and `example_R2.fastq.gz`). The following commands process this information into files that will be used for read mapping: 

In [ ]:
import glob
import pandas as pd
import re

# assumes reads are saved in data/reads/
mgx = glob.glob('data/reads/*')
mgx = [f.replace('_R1', '_R*') for f in mgx]
mgx = [f.replace('_R2', '_R*') for f in mgx]
mgx = sorted(list(set(mgx)))

# generate list of samples
samples = [f.split('/')[-1].replace('.trim.fastq.gz', '').replace('_R*', '') for f in mgx]
with open('data/samples.txt', 'w') as f:
    f.write('\n'.join(samples))

# align metadata to sample list
meta = mg.data.metadata.query('`Sample ID` in @samples').sort_values('Sample ID').reset_index(drop=True)
meta.to_csv('data/metadata.tsv', index=False, sep='\t')

# 2. Mapping reads

## Building a non-redundant set of reference marker gene sequences

To infer our strain genotypes, we use the following marker genes: _dnaG_, _rpoB_, and _gyrB_. To extract these marker genes from all representative genomes in UHGG we use the following commands:

In [ ]:
import os, sys
sys.path.append(os.path.abspath('../../mgxevo/src/'))
import mgxevo as mg

In [ ]:
%%bash -s "$mg.tl.uhgg.script_path"

# get sequences
genes=("dnaG" "gyrB" "rpoB")
for gene in ${genes[@]}; do
    python $1/1.get_fasta.py --gene $gene --multi --mgyg ./data/uhgg/genomes-nr.mgyg_ids.txt --uhgg ./data/uhgg --out ./data/uhgg/marker_gene/${gene}.fst
done

To cluster these sequences into non-redundant sets at 90% identity, we use USEARCH. Please note that USEARCH _may_ give different results depending on the how it is compiled and the machine architecture. For more information, please refer to the [USEARCH website](https://www.drive5.com/usearch/manual/reproduce.html). For reproducibility, we have provided the clustering results used in our analyses (_e.g._, see `assets/dnaG.fst` and `assets/dnaG.uc`) 

In [ ]:
%%bash -s "$mg.tl.uhgg.script_path"

genes=("dnaG" "gyrB" "rpoB")
base_dir="./data/uhgg/marker_gene"

# run USEARCH
for gene in ${genes[@]}; do
    usearch11.0.667_i86linux32 -cluster_fast ${base_dir}/${gene}.fst -id 0.9 -sort length -centroids ${base_dir}/${gene}-nr.fst -uc ${base_dir}/${gene}.uc
    rm ${base_dir}/${gene}-nr.fst.tmp
done

# concat into a single fasta
cat ${base_dir}/{dnaG,gyrB,rpoB}-nr.fst > ${base_dir}/all-nr.fasta

# generate an intermediary "gene file" for read mapping
python $1/2.make_gene_file.py ${base_dir}/all-nr.fasta > ${base_dir}/all-nr.gene.txt

## Mapping reads against reference marker genes

To generate gene alignments, we use `bowtie2` and filter the resulting alignments (see **Methods**). The resulting alignment is saved in a "pickled" `numpy` array of dimensions _(M, L, 5)_, where:

```
M = number of samples
L = number of alignment positions (alignment length)
5 = nucleotides (A,C,G,T,N)
```
More specifically, each entry $(i, j, k)$ represents how often you observe nucleotide $k$ at position $j$ in sample $i$ of your alignment. Note that this step is slow. For convenience, we provide some examples of pickle outputs in the folder `assets/pickles`.

In [ ]:
%%bash -s "$mg.tl.map.script_path"

# build bowtie2 database
bowtie2-build ./data/uhgg/marker_gene/all-nr.fasta data/all-nr > /dev/null 2>&1

# run bowtie and filter results
# assumes FASTQs are in a folder called reads
python $1/1.map_reads.py \
    --index ./data/all-nr \
    --fastq_dir ./data/reads \
    --outdir ./write/ \
    --sample_file ./data/samples.txt \
    --gene_file ./data/uhgg/marker_gene/all-nr.gene.txt

# 3. Phylogenies

## Marker gene panel

To build our marker gene panel, we first generate a gene counts matrix. Next, we parse our `USEARCH` clustering results to map alleles onto their non-redundant representative sequences. Together, this allows us to select the marker genes alleles that the most correlated across our metagenomes (see **Methods**). Since we add a small amount of random noise, the resulting panel may change each time you run this script.  For reproducibility, we have provided the panel results used in our analyses (`assets/ids.multigene.txt'`)

In [ ]:
import os, sys
sys.path.append(os.path.abspath('../../mgxevo/src/'))
import mgxevo as mg
import pandas as pd
from tqdm import tqdm

# make a folder called abundance
os.makedirs('abundance', exist_ok=True)

In [ ]:
%%bash -s "$mg.tl.marker.script_path" "$mg.data.metadata_path"

# build abundance/counts matrix
python $1/1.get_bowtie_abundances.py \
    --species_path ./write/kpileup/ \
    --samples_path ./data/samples.txt \
    --output_path ./abundance

# parse USEARCH results to map allels onto rep seqs
python $1/2.map_ids.py \
    --uclust_path ./data/uhgg/marker_gene/ \
    --genes dnaG gyrB rpoB \
    --output_file ./write/gene_ids.txt

# use the above data to select marker gene alleles that best supported 
Rscript $1/3.select_ids.r \
    --abundance_file ./write/abundance/bowtie_abundances.csv \
    --metadata $2 \
    --map_ids ./write/gene_ids.txt \
    --genes dnaG gyrB rpoB \
    --uhgg_catalogue ./data/uhgg/uhgg_catalogue \
    --uhgg_metadata ./data/uhgg/genomes-nr_metadata.tsv \
    --output_dir ./write/

## Maximum likelihood phylogenetic inference

### Single-gene phylogenies
To build single-gene phylogenies, we run `IQTree2` on consensus sequences from our alignment (again, see **Methods**). The resulting phylogenies are stored in the `write` folder.

In [ ]:
# load marker panel and extract all markers
ids = pd.read_table('write/ids.multigene.txt')
markers = sorted(list(set(ids[['dnaG', 'gyrB', 'rpoB']].values.flatten())))

# write this list of markers to file
with open('write/marker_gene_ids.txt', 'w') as f:
    f.write('\n'.join(markers))
    
iterate over these markers to build phylogenies
for marker in tqdm(markers):
    mg.tl.tree.snp_util.get_phylogeny_from_pickle(
        pickle_file=f'./write/kpileup/{marker}.aln.pickle',
        gene_file='./data/uhgg/marker_gene/all-nr.gene.txt',
        dx_file='./data/metadata.tsv',
        threads=1,
    )

### Multi-gene phylogenies

To build multi-gene phylogenies, we use a partition model in `IQTree2` along with the resulting output from our single-single gene phylogenies.

In [ ]:
# define panel
marker_genes = ['dnaG', 'gyrB', 'rpoB']

# save the genomes we want to build phylogenies for
genomes = sorted(ids['genome'].unique())
with open('write/genomes.txt', 'w') as f:
    f.write('\n'.join(genomes))

# generate symlinks 
mg.ut.create_symlinks(base_dir=os.getcwd(), 
                      source_files='./write/*fas.gb',
                      destination_dir='./data')

# concat FASTAs for downstream analyses
for _, row in tqdm(multigene_table.iterrows(), total=multigene_table.shape[0]):
    gene_ids = row[marker_genes].values
    mg.tl.tree.snp_util.process_fna_gb(
        fasta_path='./data/',
        gene_ids=gene_ids,
        name=row['genome'],
    )

# define a wrapper function that we can parallelise
def process_genome(row):
    gene_ids = row[1][marker_genes].values
    mg.tl.tree.snp_util.multigene_phylo(
        fasta_path='./data/',
        gene_ids=gene_ids,
        fasta_ext = 'fas.gb',
        name=row[1]['genome'],
        output='./write/multigene_phylo',
        threads=1,
    )
    return

# execute jobs in parallel
with ThreadPoolExecutor(max_workers=360) as executor:
    list(tqdm(executor.map(process_genome, ids.iterrows()), total=ids.shape[0]))

Note that you can also provide a `seed` for reproducibility. For example, to generate the _B. intestinalis_ phylogeny from **Figure 2A** you can use `seed = 676418`, along with the example FASTAs provided in the `assets` folder:

In [ ]:
# load panel for GUT_GENOME001255
gene_ids = ids.loc[ids.genome=='GUT_GENOME001255'][marker_genes].values

# generate tree 
# note that the *.processed.fas.gb were generated from the single gene phylogeny step
mg.tl.tree.snp_util.multigene_phylo(
    fasta_path='./assets/',
    gene_ids=gene_ids,
    name=row[1]['genome'],
    output='./write/multigene_phylo',
    threads=1,
    seed= 676418
)

## Integration of reference genomes

### Identifying sequences to be integrated into phylogenies
To integrate reference genomes into our phylogenies, we first BLAST our template sequences (_i.e._, the sequences used to generate our phylogenies) against all marker genes from the entire UHGG catalog. The resulting BLAST hits are then filtered and processed. Specifically, we convert them to FASTA files where each file contains reference sequences to be integrated into their respective phylogenies.

In [ ]:
# write template sequences to file
x = ids[marker_genes]
x = x.melt(value_name='gene').dropna()['gene'].unique()
f = mg.ut.seq_util.read_fst("./data/uhgg/marker_gene/all-nr.fasta")
mg.ut.seq_util.dict2fasta({k: f[k] for k in x}, './write/all.tree-seeds.fasta')

In [ ]:
%%bash -s "$mg.tl.marker.script_path" 

# write marker genes from entire UHGG catalog to file
python $1/4.merge_mgyg.py \
    --multigene_table ./write/ids.multigene.txt \
    --uhgg_dir ./data/uhgg/ \
    --genes dnaG gyrB rpoB \
    --output_file ./write/pangenome_markers.fasta

# make BLAST database
makeblastdb -in ./write/pangenome_markers.fasta -dbtype nucl > ./tmp/makeblastdb.log

# run BLAST 
blastn -query ./write/all.tree-seeds.fasta \
    -db ./write/pangenome_markers.fasta \
    -outfmt 6 \
    -evalue 1e-5 \
    -max_target_seqs 100000000 \
    -num_threads 30 \
    -out ./write/tree-seeds.mgyg.b6 > ./tmp/blastn.log

# filter blast results to keep only HQ matches (see methods)
python $1/5.filter_blast.py \
    --fasta ./write/all.tree-seeds.fasta \
    --blast ./write/tree-seeds.mgyg.b6 \
    --filtered_blast ./write/tree-seeds.mgyg.filtered.b6

# write to file (single gene phylogenies)
mkdir -p ./write/marker_reference/
python $1/6.blast2fna_single.py \
    --filtered_blast ./write/tree-seeds.mgyg.filtered.b6 \
    --multigene_table ./write/ids.multigene.txt \
    --pangenome_marker ./write/pangenome_markers.fasta \
    --genes dnaG gyrB rpoB \
    --output_dir ./write/marker_reference/

# process results for multigene phyo (keep only marker gene hits from same genome)
Rscript $1/7.proc_refmap.r \
    --multigene_table ./write/ids.multigene.txt \
    --blast_results ./write/tree-seeds.mgyg.filtered.b6 \
    --marker_mapping ./write/marker_reference/tree-seeds.mgyg.filtered.ref_map.txt \
    --output_dir ./write/marker_reference/

# write and concat seqs to file (multigene phlyo)
python $1/8.write_refs.py \
    --refmap_proc_file ./write/marker_reference/tree-seeds.mgyg.filtered.ref_map.proc.txt \
    --tree_seq_file ./write/pangenome_markers.fasta \
    --output_dir ./write/marker_reference/multigene

### Integration of sequences into phylogenies

Having identified which sequences to integrate into our phylogenies, we now use [`UShER`](https://usher-wiki.readthedocs.io) to integrate these sequences into our trees.

In [ ]:
%%bash -s "$mg.tl.tree.script_path"

# single gene 
cat ./write/marker_gene_ids.txt | parallel --progress -j 60 "python $1/add_seqs2tree.py \
    --tree ./write/{}.tree \
    --name {} \
    --fasta ./write/{}.fas.gb \
    --gtf ../data/uhgg/marker_gene/all-nr.gene.txt \
    --seq ./write/marker_reference/{}.ref.fna \
    --output ./write/{}.ref.tree"

# multigene 
cat ./write/genomes.txt | parallel --progress -j 350 "python $1/add_seqs2tree.multi.py \
    --tree ./write/multigene_phylo/{}.tree \
    --name {} \
    --fasta ./data/{}.fas.conaln.gb \
    --seq ./write/marker_reference/multigene/{}.refseq.fna \
    --multigene_table ./write/ids.multigene.txt \
    --refseq ./data/uhgg/marker_gene/all-nr.fasta \
    --genes dnaG gyrB rpoB \
    --output ./write/multigene_phylo/{}.ref.tree"

# 4. Analyses

## Enrichment

The following code blocks allow us to test for phylogenetic enrichment (while controlling for covariates such as batch, **Methods**), while also testing for differential abundance.

### Phylogenetic enrichment

In [ ]:
import os, sys
sys.path.append(os.path.abspath('../../mgxevo/src/'))
import mgxevo as mg

In [ ]:
%%bash -s "$mg.tl.enrich.script_path" "$mg.data.metadata_path"

# enrichment for single gene
cat write/marker_gene_ids.txt | parallel --progress -j 28 "Rscript $1/phylo.r \
    --name {} \
    --tree_file ./write/{}.tree \
    --metadata $2 \
    --output_dir ./write/single

# enrichment for multigene gene
cat ./write/genomes.txt | parallel --progress -j 28 "Rscript $1/phylo.r \
    --name {} \
    --tree_file ./write/multigene_phylo/{}.tree \
    --metadata $2 \
    --output_dir ./write/multigene_phylo

### Differential abundance

In [ ]:
%%bash -s "$mg.tl.enrich.script_path" "$mg.data.metadata_path"

Rscript $1/abund.r \
    --bowtie_aln ./abundance/bowtie_abundances.csv \
    --metadata $2 \
    --multigene_table ./write/ids.multigene.txt \
    --output_dir ./write/ \
    --genes dnaG gyrB rpoB \

## Strain abundance estimates

To estimate the abundances of strains, we use [StrainFinder](https://github.com/cssmillie/StrainFinder). To begin we first create a new `conda` environment for StrainFinder. We then select strains (see **Methods**) and convert them into a `numpy` array that we can input into StrainFinder. Finally, we run StrainFinder.

In [ ]:
%%bash -s "$mg.tl.strainfinder.script_path" "$mg.data.metadata_path"

# create conda environment
conda env create -f $1/environment.yml
conda activate strainfinder

# select strains to analyse 
Rscript $1/1.select_strains.r \
    --multigene_table ./write/ids.multigene.txt \
    --tree_dir ./write/multigene_phylo/ \
    --phylo_enrichment_dir ./write/multigene_phylo \
    --output_dir ./write/

# concat into numpy for StrainFinder 
mkdir -p ./write/multigene_np
python $1/2.merge_np.py \
    --strains ./write/genomes.use.txt \
    --aln_pickle_dir ./write/kpileup/ \
    --multigene_table ./write/ids.multigene.txt \
    --genes dnaG gyrB rpoB \
    --output_dir ./write/multigene_np

# actually run strainfinder
mkdir -p ./write/strainfinder
for genome in (cat ./write/genomes.use.txt); do
    python $1/3.run_sf.py \
        --genome $genome \
        --aln ./write/multigene_np/$genome.multigene.aln.pickle \
        --tree ./write/multigene_phylo/$genome.tree \
        --samples ./data/samples.txt \
        --metadata $2 \
        --final_tips ./write/strainfinder.final_tips.txt \
        --output ./write/strainfinder/$genome.sf.txt
        --use_phylo 1
done

By default, this only estimates strain abundances for samples that are in the multigene phylogeny. To generate estimates for all samples in the alignment use `--use_phylo 0`. This might be useful when estimating strain abundances for longitudinal samples (see below).

#### Summarize results
For convenience, we can summarize all this information. This will produce an `RDS` file (`res.rds`), a table of strain frequencies (`strain.freq.csv`), and a text file summarizing our tree enrichment statistics (`stats.summary.txt`). However, this step can be slow.

In [ ]:
%%bash -s "$mg.tl.strainfinder.script_path" "$mg.data.metadata_path"

# first activate previous conda environment 
conda activate mgxevo

# run script
Rscript $1/4.summarise.r \
    --multigene_table ./write/ids.multigene.txt \
    --tree_dir ./write/multigene_phylo/ \
    --metadata $2 \
    --uhgg_metadata ./data/uhgg/genomes-all_metadata.tsv \
    --sf_dir ./write/strainfinder \
    --phylo_enrichment_dir ./write/multigene_phylo \
    --output_dir ./write/ \
    --genomes ./write/genomes.use.txt \
    --strainfinder_tips ./write/strainfinder.final_tips.txt

#### _In silico_ strain competition experiments

To perform an _in silico_ strain competition experiment we can leverage our longitudinal data. To begin, we first re-run StrainFinder:

In [ ]:
mkdir -p ./write/strainfinder
for genome in (cat ./write/genomes.use.txt); do
    python $1/3.run_sf.py \
        --genome $genome \
        --aln ./write/multigene_np/$genome.multigene.aln.pickle \
        --tree ./write/multigene_phylo/$genome.tree \
        --samples ./data/samples.txt \
        --metadata $2 \
        --final_tips ./write/strainfinder.final_tips.txt \
        --output ./write/strainfinder/$genome.sf.long.txt
        --use_phylo 0
done

We can now load Table S1 and select the appropriate time points for our strain competition experiment (see **Methods**):

In [ ]:
%%bash -s "$mg.tl.strainfinder.script_path" "$mg.data.metadata_path"

# first activate previous conda environment 
conda activate mgxevo

# run script
Rscript $1/5.compete_strains.r \
    --multigene_table ./write/ids.multigene.txt \
    --bowtie_aln ./abundance/bowtie_abundances.csv \
    --metadata $2 \
    --sf_dir ./write/strainfinder \
    --phylo_enrichment_dir ./write/multigene_phylo \
    --output_dir ./write/ \
    --genomes ./write/genomes.use.txt \

## Fecal calprotectin predictions

The following code runs both our Random Forest and GBM models over a wide range of parameters, saving the combined output in an RDS file. 

In [ ]:
%%bash -s "$mg.tl.gene_strain.script_path" "$mg.data.metadata_path"

# run RF and GBM models over a wide range of parameters (with n=100 replicates)
for a in {0,25,50}; do
    for b in {500,1000,2000}; do 
        for c in {1,2}; do 
            for n in {1..100}; do 
                Rscript $1/run_fcp_models.r \
                    --metadata $2 \
                    --panel ./write/ids.multigene.txt \
                    --nr ./write/multigene.nr.txt \
                    --abd ./write/abundance/bowtie_abundances.csv \
                    --freqs ./write/strain.freq.csv \
                    --phylo_enrichment ./write/multigene_phylo \
                    --cut1 0.$a \
                    --cut2 $b \
                    --ibd FALSE \
                    --fopt $c \
                    --coh sdf \
                    --outdir ./write/ \
                    --out res.$a.$b.FALSE.$c.prism.$n.rds; 
            done;
        done;
    done;
done;

## Gene strain inference

To perform our gene-strain inference, we first iterate through our phylogenies to identify genomes that have a sufficient number of reference genomes to infer the content of our disease-adapted strains. Next, for each reference genome that integrates into a disease-adapted node, we store the relevant pan-genome presence-absence matrices in an RDS file called `*.mtx.rds`. Finally, we run our enrichment model on each of these matrices.

In [ ]:
%%bash -s "$mg.tl.gene_strain.script_path"

# first identify genomes that we can use
Rscript $1/1.select_ids.r \
    --uhgg_dir ./data/uhgg/ \
    --info ./write/stats.summary.txt \
    --phylo_dir ./write/multigene_phylo \
    --outdir ./write/

# merge pan-genomes into a convenient RDS file
tail -n +2 ./write/ids.use.txt | cut -f1 | parallel --progress -j 30 "Rscript $1/2.build_mtx.r \
    --genome {} \
    --gene_strain_ids ./write/ids.use.txt \
    --uhgg_dir ./data/uhgg/ \
    --phylo_dir ./write/multigene_phylo/ \
    --outdir ./write/"

# run enrichment script
tail -n +2 ./write/ids.use.txt | cut -f1 | parallel --progress -j 30 "Rscript $1/3.enrichment.r \
    --genome {} \
    --gene_strain_ids ./write/ids.use.txt \
    --uhgg_dir ./data/uhgg/ \
    --mtx_dir ./write/ \
    --outdir ./write/"

### DIAMOND
To validate our gene-strain predictions, we use [DIAMOND](https://github.com/bbuchfink/diamond) to obtain gene abundance estimates, specifically focusing on genes that occur in >100 samples.

In [ ]:
import os
import glob
from tqdm import tqdm
import pandas as pd

# get a list of all FASTQ files
all_fastq = glob.glob('../2.map_reads/data/reads/*.trim.fastq.gz')
all_fastq = sorted([os.path.basename(x).replace('.trim.fastq.gz', '') for x in all_fastq])

with open('data/fastqs.txt', 'w') as f:
    f.write('\n'.join(all_fastq))

In [ ]:
%%bash -s "$mg.tl.gene_strain.script_path" 

# run DIAMOND
mkdir -p ./write/diamond
cat data/fastqs.txt | parallel --progress -j 35 "python $1/1.run_diamond.py \
    -q ../2.map_reads/data/reads/{}.trim.fastq.gz \
    -o ./write/diamond/{} \
    -d ../1.uhgg_db/data/uhgg/uhgp-90/uhgp-90_hq-diamond-db.dmnd \
    --threads 10"

# parse output 
mkdir -p ./write/counts
cat data/fastqs.txt | parallel --progress -j 360 "python $1/2.blast2counts.py \
    --blast ./write/diamond/{}.gz \
    --pct 90 \
    --evalue 1e-8 \
    --output ./write/counts/{}"

# get genes present in >100 samples
for f in ./write/counts/*.counts.txt
do 
    head -n 1 $f
done >> ./write/genes.txt

cat ./write/genes.txt | tr '\t' '\n' | awk '//{x[$0]++}END{for(k in x){print(k, x[k])}}' > ./write/gene_counts.txt

awk '$2 > 100' ./write/gene_counts.txt | cut -d' ' -f1 > ./write/genes.over100.txt

# convert counts to RDS files for downstream analyses
mkdir -p ./write/diamond_mtx
seq 1 50 | parallel --progress -j 50 Rscript $1/3.get_diamond_mtx.r \
    -i {} \
    -n 50 \
    -g ./write/genes.over100.txt \
    --count_dir ./write/counts \
    --out_dir ./write/diamond_mtx

Finally, to focus on genes associated with our taxa of interest, we can subset our DIAMOND output:

In [ ]:
%%bash -s "$mg.tl.gene_strain.script_path" "$mg.data.metadata_path"

tail -n +2 ./write/ids.use.txt | cut -f1 | parallel --progress -j 30 "Rscript $1/4.get_dmd_counts_mtx.r \
    --genome {} \
    --metadata $2 \
    --gene_strain_ids ./write/ids.use.txt \
    --uhgg_dir ./data/uhgg/ \
    --phylo_dir ./write/multigene_phylo/ \
    --dmd_dir ./write/diamond_mtx/ \
    --outdir ./write/"

### Putative _E. lenta_ gene predictions

To identify gene groups that are co-abundant with _E. lenta_ abundances, we first generate a table of _E. lenta_ abundance; and a list of core _E. lenta_ genes.

In [ ]:
%%bash -s "$mg.tl.gene_strain.script_path" "$mg.data.metadata_path"

Rscript $1/1.get_elenta_ab.r\
    --metadata $2 \
    --panel ./write/ids.multigene.txt \
    --abd ./write/abundance/bowtie_abundances.csv \
    --strain_freq ./write/strain.freq.csv \
    --phylo_enrichment ./write/multigene_phylo \
    --uhgg_dir ./data/uhgg/ \
    --outdir ./write/

We next correlate our gene abundances from DIAMOND against these _E. lenta_ abundances, and use these to make predictions about which genes _may_ be associated with _E. lenta_:

In [ ]:
%%bash -s "$mg.tl.gene_strain.script_path" "$mg.data.metadata_path"

# 1) first calcualte the row sum for our diamond matrix
Rscript $1/2.calc_dmd_rowSum.r\
    --metadata $2 \
    --panel ./write/ids.multigene.txt \
    --abd ./write/abundance/bowtie_abundances.csv \
    --dmd_dir ./write/diamond_mtx/ \
    --outdir ./write/

# 2) calculate correlations on chunked matricies
seq 1 50 | parallel --progress -j 50 Rscript $1/3.calc_elenta_cor.r \
    --metadata $2 \
    --el_abd ./write/el.ab.tsv \
    --dmd_dir ./write/diamond_mtx/ \
    --gene100 ./write/genes.over100.txt \
    --outdir ./write/ \
    --chunk {}

# 3) merge correlation output
Rscript $1/4.merge_el_cor.r \
    --outdir ./write/

# 4) predict
Rscript $1/5.predict_elenta.r \
    --outdir ./write/

Having identified a set of putative _E. lenta_ genes, we can now subset our DIAMOND matrix and run a linear model to identify genes that may possibly associate with health- or disease-associated strains.

In [ ]:
%%bash -s "$mg.tl.gene_strain.script_path" "$mg.data.metadata_path" "$mg.tl.uhgg.script_path"

# 6) subset DIAMOND to predicted E. lenta genes
Rscript $1/6.subset_diamond.r \
    --meta $2 \
    --gene ./write/el.predicted_genes.txt \
    --dmd_dir ./write/diamond_mtx/ \
    --outdir ./write/ \
    --outfile elenta.rds

# 7) run a linear regression model and add KEGG and EggNOG annotations
Rscript $1/7.run_elenta_de.r \
    --metadata $2 \
    --abd ./write/el.ab.tsv \
    --uhgp_tools $3 \
    --dmd ./write/elenta.rds \
    --phylo_enrichment ./write/multigene_phylo \
    --uhgg_dir ./data/uhgg/uhgp90/ \
    --tree_dir  ./write/multigene_phylo/ \
    --outdir /write/